In [1]:
! pip install -r ./requirements.txt

In [2]:
from langchain.document_loaders import PDFPlumberLoader, DirectoryLoader

In [3]:
loader = DirectoryLoader("./DEMO/", glob="**/*.pdf", loader_cls=PDFPlumberLoader)

In [4]:
docs = loader.load()

In [5]:
len(docs)

404

In [6]:
for doc in docs:
    print(doc.metadata)

{'source': 'DEMO/thanhnien_travel_articles.pdf', 'file_path': 'DEMO/thanhnien_travel_articles.pdf', 'page': 0, 'total_pages': 64, 'Author': '(anonymous)', 'CreationDate': "D:20240909103122-07'00'", 'Creator': '(unspecified)', 'Keywords': '', 'ModDate': "D:20240909103122-07'00'", 'Producer': 'ReportLab PDF Library - www.reportlab.com', 'Subject': '(unspecified)', 'Title': '(anonymous)', 'Trapped': 'False'}
{'source': 'DEMO/thanhnien_travel_articles.pdf', 'file_path': 'DEMO/thanhnien_travel_articles.pdf', 'page': 1, 'total_pages': 64, 'Author': '(anonymous)', 'CreationDate': "D:20240909103122-07'00'", 'Creator': '(unspecified)', 'Keywords': '', 'ModDate': "D:20240909103122-07'00'", 'Producer': 'ReportLab PDF Library - www.reportlab.com', 'Subject': '(unspecified)', 'Title': '(anonymous)', 'Trapped': 'False'}
{'source': 'DEMO/thanhnien_travel_articles.pdf', 'file_path': 'DEMO/thanhnien_travel_articles.pdf', 'page': 2, 'total_pages': 64, 'Author': '(anonymous)', 'CreationDate': "D:20240909

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)

In [8]:
chunks = splitter.split_documents(docs)
len(chunks)

1138

In [9]:
len(chunks[0].page_content)

969

In [10]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large",
    model_kwargs={"device": device},
)

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no lon

In [11]:
embeddings.embed_query(chunks[0].page_content)

[-0.03873845934867859,
 -0.006944548338651657,
 -0.027697298675775528,
 0.05117695406079292,
 -0.0003282633551862091,
 0.02346440777182579,
 0.009324850514531136,
 0.005106600932776928,
 -0.037926092743873596,
 0.01918814703822136,
 0.029349369928240776,
 -0.008915056474506855,
 0.00257931649684906,
 0.05005127564072609,
 0.0038384299259632826,
 -0.009979170747101307,
 -0.046253226697444916,
 0.016126928851008415,
 -0.04009188339114189,
 0.006927108392119408,
 0.0689050704240799,
 0.018936503678560257,
 -0.008779169991612434,
 0.04503055289387703,
 0.002132278634235263,
 0.0018204329535365105,
 -0.02544146589934826,
 -0.001202915795147419,
 0.03451726958155632,
 -0.06212156265974045,
 0.053002383559942245,
 -0.0488973967730999,
 -0.05078906565904617,
 -0.04290248453617096,
 -0.015319800935685635,
 0.026457853615283966,
 -0.011709481477737427,
 0.04187354817986488,
 0.03688235580921173,
 0.05533836781978607,
 0.005070528946816921,
 0.03607160225510597,
 -0.008578496053814888,
 -0.056355

In [12]:
from langchain_community.vectorstores.chroma import Chroma

store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)

In [13]:
question = "Xem pháo hoa ở đâu?"

In [14]:
retriever = store.as_retriever()

In [15]:
retrieved_docs = retriever.invoke(question)

In [16]:
for doc in retrieved_docs:
    print('----')
    print(doc.page_content)

----
lượng đồ ăn của quán luôn ổn định.
Khách đến ăn thường ngồi ở vỉa hè gần quán. Bát, thìa, chỗ ngồi sau khi khách ăn xong
luôn được nhân viên quán dọn ngay để đảm bảo vệ sinh và khách mới có chỗ ngồi.
Tú Nguyễn
----
xây dựng những chính sách phù hợp. Phải có những chính sách mang tầm chiến lược
quốc gia, từ định vị thương hiệu điểm đến tới thúc đẩy mạnh kinh tế đêm, mở toang liên
kết vùng. Nếu cứ định hướng chung chung thì không thể làm được và những nghịch lý
như khách đến đông, khách sạn vẫn lỗ, hệ sinh thái du lịch vẫn yếu… sẽ còn tiếp diễn.
Ông Phạm Hà
----
xây dựng những chính sách phù hợp. Phải có những chính sách mang tầm chiến lược
quốc gia, từ định vị thương hiệu điểm đến tới thúc đẩy mạnh kinh tế đêm, mở toang liên
kết vùng. Nếu cứ định hướng chung chung thì không thể làm được và những nghịch lý
như khách đến đông, khách sạn vẫn lỗ, hệ sinh thái du lịch vẫn yếu… sẽ còn tiếp diễn.
Ông Phạm Hà
----
nói.
Ông Hồ Văn Khu, Chủ tịch xã Phước Năng, cho biết thỉnh thoảng có một số

In [17]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    <s> [INST] You are a helpful assitant, answer question about ingested policy documents. Use only in the context, do not use any information outside of this context. If you don't know just say that you don't know.[/INST] </s> 
    [INST] Question: {question} 
    Context: {context} 
    Answer: [/INST]
    """
)

In [18]:
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="phi3", base_url='http://127.0.0.1:11434')

In [19]:
from langchain.globals import set_debug

set_debug(True)

In [20]:
from langchain.schema.runnable import RunnableConfig, RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [21]:
chain.invoke(question)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Xem pháo hoa ở đâu?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "Xem pháo hoa ở đâu?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Xem pháo hoa ở đâu?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Xem pháo hoa ở đâu?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question>] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOllam

"The documents provided do not contain any specific information about the location of a traditional meal within or around Bà Quyết Mountain in Hai Duong Province, Vietnam. The context discusses various topics related to travel and tourism without referencing this particular subject matter directly tied to ingested policy documents on such local culinary experiences at these mountains' restaurants."